# Prior estimation

In [2]:
include("src/main.jl")
OPTCONFIG=OptConfig(XTOLABS=1e-8, DEBUG=true);

In [3]:
# enable pyplot-latex
PyCall.PyDict(PyPlot.matplotlib["rcParams"])["text.usetex"] = true

true

## The Model

In [4]:
n = 200
ndata = 100
σ = .5
prior = Uniform(0,2)

Distributions.Uniform{Float64}(a=0.0, b=2.0)

In [5]:
srand(5)
m = FEModel(f=x->x, xs=linspace(0,2,n), σ=σ)
mt = FEModel(f=x->x, xs=linspace(0,1,n), σ=σ)
d = generatedata(m, prior, ndata);

In [6]:
ncv = 20
#kfold = length(d);
kfold = 10

10

In [ ]:
## er estimator
regs = [ReferenceRegularizer(m, γ) for γ in linspace(0.01,10,ncv)]
@time ss = [cvscore(m, d, d->ebprior(m, d, r), kfold) for r in regs]
plot([r.γ for r in regs], ss)
regR = regs[indmax(ss)].γ

In [ ]:
## er estimator
regs = [ReferenceRegularizer(mt, γ) for γ in linspace(0.0001,0.001,ncv)]
@time ss = [cvscore(mt, d, d->ebprior(mt, d, r), kfold) for r in regs]
plot([r.γ for r in regs], ss)
regRt = regs[indmax(ss)].γ

In [ ]:
@show regR
wr  = ebprior(m,  d, ReferenceRegularizer(m,  regR))
wrt = ebprior(mt, d, ReferenceRegularizer(mt, regR))

figure(figsize=(4,3))
plot([0,2],[0.5,.5], label="\$\\pi_{\\rm true}\$", alpha=.3, linewidth=2.2)
plot(m.xs, wr/2*n, label="\$\\pi_{\\rm er}\$")
plot(mt.xs, wrt*n, label="\$\\pi^0_{\\rm er}\$")
plot(d, zeros(d), marker="|", linestyle=" ", label="data")

ylim(0,2)

yticks([0,1,2])
xticks([0,1,2])
legend()
vlines([0,1,2])
savefig("counterex.pdf", bbox_inches="tight")